In [ ]:
!pip install rasterio

In [34]:
import torch
from torch.utils.data import Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [29]:
!ls '/content/drive/MyDrive/Work/Collaborations/NASA GSFC Sabbatical Fall 2021'


'AWS info for super resolution.gdoc'
'bathymetry super resolution project'
'Sabbatical Fall 2021 My Own Notes - NASA GSFC .gdoc'


In [24]:
!ls /content/drive/MyDrive/Work/Collaborations/NASA\ GSFC\ Sabbatical\ Fall\ 2021/bathymetry\ super\ resolution\ project/SuperRes_AOI_v2/

Tile_36_-5_EPSG4326_Etopo.tif	Tile_37_-76_EPSG4326_Gebco.tif
Tile_36_-5_EPSG4326_Gebco.tif	Tile_-40.6_173.7_EPSG4326_Etopo.tif
Tile_37_-76_EPSG4326_Etopo.tif	Tile_-40.6_173.7_EPSG4326_Gebco.tif


In [31]:
common_path = '/content/drive/MyDrive/Work/Collaborations/NASA GSFC Sabbatical Fall 2021/bathymetry super resolution project/SuperRes_AOI_v2/'
import sys
import glob
files_hr = sorted(glob.glob(common_path + "*Etopo.tif"))
print(files_hr)

['/content/drive/MyDrive/Work/Collaborations/NASA GSFC Sabbatical Fall 2021/bathymetry super resolution project/SuperRes_AOI_v2/Tile_-40.6_173.7_EPSG4326_Etopo.tif', '/content/drive/MyDrive/Work/Collaborations/NASA GSFC Sabbatical Fall 2021/bathymetry super resolution project/SuperRes_AOI_v2/Tile_36_-5_EPSG4326_Etopo.tif', '/content/drive/MyDrive/Work/Collaborations/NASA GSFC Sabbatical Fall 2021/bathymetry super resolution project/SuperRes_AOI_v2/Tile_37_-76_EPSG4326_Etopo.tif']


In [38]:
import rasterio
import numpy as np

img_hr = rasterio.open(files_hr[0])

img_hr = img_hr.read(1)
img_hr=np.float32(img_hr)
print(img_hr)

[[-162. -154. -142. ...  295.  237.  211.]
 [-165. -156. -142. ...  335.  341.  279.]
 [-164. -153. -138. ...  358.  417.  402.]
 ...
 [ 770.  570.  461. ... -205. -278. -323.]
 [ 780.  496.  448. ... -148. -173. -207.]
 [ 708.  459.  443. ... -133. -131. -139.]]


In [35]:
from sklearn.preprocessing import StandardScaler

In [39]:
scaler_f = StandardScaler()
img_hr_transformed = scaler_f.fit_transform(img_hr) 
print(img_hr_transformed)

[[-0.9205479  -0.90318877 -0.8853936  ...  2.3192701   1.857064
   1.628433  ]
 [-0.926244   -0.90727955 -0.8853936  ...  2.5845664   2.5230455
   2.0521538 ]
 [-0.9243453  -0.9011434  -0.8771841  ...  2.7371118   3.0097244
   2.8185897 ]
 ...
 [ 0.84904957  0.5776822   0.35219166 ... -0.99693376 -1.4408256
  -1.6990207 ]
 [ 0.8680367   0.4263225   0.3255107  ... -0.61888653 -0.7684403
  -0.976203  ]
 [ 0.73132956  0.35064262  0.3152488  ... -0.5194004  -0.49948618
  -0.5524823 ]]


In [40]:
img_hr_inverse_transformed = scaler_f.inverse_transform(img_hr_transformed)
print(img_hr_inverse_transformed)

[[-162.00003 -154.      -142.      ...  295.       237.       211.     ]
 [-165.      -156.      -142.      ...  335.       341.       279.00003]
 [-164.      -153.      -138.      ...  358.       417.       402.     ]
 ...
 [ 770.       570.       461.      ... -205.      -278.      -323.     ]
 [ 780.       496.       448.      ... -148.      -173.      -207.     ]
 [ 708.       459.       443.      ... -133.      -131.      -139.     ]]


In [41]:
img_hr_list = []
for index in list(files_hr):
    img_hr = rasterio.open(files_hr[0])
    img_hr = img_hr.read(1)
    img_hr=np.float32(img_hr)
    img_hr_list.append(img_hr)

In [42]:
scaler_f = StandardScaler()
img_hr_list_transformed = scaler_f.fit_transform(img_hr_list) 
print(img_hr_list_transformed)

ValueError: ignored

In [50]:
import os
#output_path = os.path.join(common_path, "Transformed_"+batch["name"][0])
with rasterio.Env():
    for file_hr in list(files_hr):
        output_file = os.path.splitext(file_hr)[0]+"Transformed2"+os.path.splitext(file_hr)[1]  
        with rasterio.open(file_hr) as src: 
            profile = src.meta
            print(profile)
            img_hr = src.read(1)
            img_hr=np.float32(img_hr)
            #should not do tranformation for each file, will cause problems during inverse_transformation.
            #img_hr_transformed = scaler_f.fit_transform(img_hr)
            img_hr_transformed = scaler_f.transform(img_hr)
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(img_hr_transformed, indexes = 1) 

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 128, 'height': 128, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.016666666666666666, 0.0, 172.625,
       0.0, -0.016666666666666663, -39.52499999999996)}
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 128, 'height': 128, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.016666666666666666, 0.0, -6.074999999999989,
       0.0, -0.016666666666666663, 37.07500000000002)}
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 128, 'height': 128, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.016666666666666666, 0.0, -77.07499999999999,
       0.0, -0.016666666666666663, 38.07500000000002)}


In [51]:

transformed_files_hr = sorted(glob.glob(common_path + "*EtopoTransformed2.tif"))
print(transformed_files_hr)
with rasterio.Env():
    for file_hr in list(transformed_files_hr):
        output_file = os.path.splitext(file_hr)[0]+"DeTraned2"+os.path.splitext(file_hr)[1]  
        with rasterio.open(file_hr) as src: 
            profile = src.meta
            print(profile)
            img_hr = src.read(1)
            img_hr=np.float32(img_hr)
            img_hr_detransformed = scaler_f.inverse_transform(img_hr)
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(img_hr_detransformed, indexes = 1) 

['/content/drive/MyDrive/Work/Collaborations/NASA GSFC Sabbatical Fall 2021/bathymetry super resolution project/SuperRes_AOI_v2/Tile_-40.6_173.7_EPSG4326_EtopoTransformed2.tif', '/content/drive/MyDrive/Work/Collaborations/NASA GSFC Sabbatical Fall 2021/bathymetry super resolution project/SuperRes_AOI_v2/Tile_36_-5_EPSG4326_EtopoTransformed2.tif', '/content/drive/MyDrive/Work/Collaborations/NASA GSFC Sabbatical Fall 2021/bathymetry super resolution project/SuperRes_AOI_v2/Tile_37_-76_EPSG4326_EtopoTransformed2.tif']
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 128, 'height': 128, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.016666666666666666, 0.0, 172.625,
       0.0, -0.016666666666666663, -39.52499999999996)}
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 128, 'height': 128, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.016666666666666666, 0.0, -6.074999999999989,
       0.0, -0.016666666666666663, 37.0750000000